## Social Network Ads - Logical Regression

### Social Network Advertisement 

Context
=========
There's a story behind every dataset and here's your opportunity to share yours.

Content
========
What's inside is more than just rows and columns. Make it easy for others to get started by describing 
how you acquired the data and what time period it represents, too.

Acknowledgements
===================
We wouldn't be here without the help of others. If you owe any attributions or thanks, include them here along with any citations of past research.

Inspiration
============
Your data will be in front of the world's largest data science community. What questions do you want to see answered?

In [ ]:
# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing Pandas and NumPy
import pandas as pd, numpy as np

In [ ]:
# Importing all datasets
social_ads = pd.read_csv("/kaggle/input/social-network-ads/Social_Network_Ads.csv")
social_ads.head()

In [ ]:
social_ads.Purchased.value_counts()

In [ ]:
social_ads.shape

In [ ]:
# let's look at the statistical aspects of the dataframe
social_ads.describe()

In [ ]:
# Let's see the type of each column
social_ads.info()

### Step 3: Data Preparation

In [ ]:
social_ads.dtypes

#### Converting some binary variables (Yes/No) to 0/1

In [ ]:
# List of variables to map

varlist =  ['Gender']

# Defining the map function
def binary_map(x):
    return x.map({"Male": 1, "Female": 0})

# Applying the function to the housing list
social_ads[varlist] = social_ads[varlist].apply(binary_map)

In [ ]:
social_ads.head(2)

In [ ]:
social_ads.dtypes

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ["User ID","Age","EstimatedSalary"]

social_ads[num_vars] = scaler.fit_transform(social_ads[num_vars])

social_ads.head()

### Checking if there is any null values 

In [ ]:
social_ads.isnull().sum()

In [ ]:
# Checking the percentage of missing values
round(100*(social_ads.isnull().sum()/len(social_ads.index)), 2)

From , the `Above Dataset`, the max-min scaler is used to put all the values between 0 and 1

### Checking for Outliers

In [ ]:
# Checking for outliers in the continuous variables
num_social_ads = social_ads[["User ID","Gender","Age","EstimatedSalary","Purchased"]]

In [ ]:
# Checking outliers at 25%, 50%, 75%, 90%, 95% and 99%
num_social_ads.describe(percentiles=[.25, .5, .75, .90, .95, .99])

## Step 4: Test-Train Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = social_ads
X.head()

In [ ]:
# Putting response variable to y
y = social_ads['EstimatedSalary']

In [ ]:
y.head()
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
EstimatedSalary = (sum(social_ads['EstimatedSalary'])/len(social_ads['EstimatedSalary'].index))*100
EstimatedSalary

## Building our model

This time, we will be using the **LinearRegression function from SciKit Learn** for its compatibility with RFE (which is a utility from sklearn)

### RFE
Recursive feature elimination

In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
# Running RFE with the output number of the variable equal to 10
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 5)             # running RFE
rfe = rfe.fit(X_train, y_train)

In [ ]:
rfe.support_

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
col = X_train.columns[rfe.support_]
col

In [ ]:
X_train.columns[~rfe.support_]

### Step 6: Looking at Correlations

In [ ]:
# Importing matplotlib and seaborn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Let's see the correlation matrix 
plt.figure(figsize = (8,5))        # Size of the figure
sns.heatmap(social_ads.corr(),annot = True)
plt.show()

### Step 7: Model Building
Let's start by splitting our data into a training set and a test set.

#### Running Your First Training Model

In [ ]:
import statsmodels.api as sm

In [ ]:
# Logistic regression model
logm1 = sm.GLM(y_train,(sm.add_constant(X_train)), family = sm.families.Binomial())
logm1.fit().summary()